In [ ]:
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver

# -------- Define State --------
class workflowState(TypedDict):
    preferences: str
    breakfast: list[str]
    lunch: list[str]
    dinner: list[str]

# -------- Init LLM --------
# llm = init_chat_model("google_genai:gemini-2.0-flash")

# -------- Nodes --------
def get_preferences(state: workflowState):
    state["preferences"] = "vegetarian"
    return {"preferences": state["preferences"]}

def gen_breakfast(state: workflowState):
    # result = llm.invoke(f"Generate {state['preferences']} breakfast options")
    user_input1 = input("Enter breakfast list: ")
    # state["breakfast"] = ["idly", "upma"]
    print(f"{[breakfast]}-------------")
    state["breakfast"] = [item.strip() for item in user_input1.split(",")]
    return {"breakfast": state["breakfast"]}

def gen_lunch(state: workflowState):
    user_input2 = input("Enter lunch list: ")
    # state["lunch"] = ["rice", "dal", "vegetable curry"]
    state["lunch"] = [item.strip() for item in user_input2.split(",")]
    return {"lunch": state["lunch"]}

def gen_dinner(state: workflowState):
    user_input3 = input("Enter dinner list: ")
    # state["dinner"] = ["chapati", "paneer"]
    state["dinner"] = [item.strip() for item in user_input3.split(",")]
    return {"dinner": state["dinner"]}

def review_meal(state: workflowState, meal_key: str, next_node: str):
    print(f"\nReview {meal_key}: {state[meal_key]}")
    decision = interrupt(f"Approve {meal_key}? Say 'yes' to approve, or anything else to regenerate.")
    if decision == "yes":
        return Command(goto=next_node)
    else:
        return Command(goto=f"gen_{meal_key}")

def review_bf(state: workflowState):
    return review_meal(state, "breakfast", "final_bf")

def review_lunch(state: workflowState):
    return review_meal(state, "lunch", "final_lunch")

def review_dinner(state: workflowState):
    return review_meal(state, "dinner", "final_dinner")

def final_bf(state: workflowState):
    return state

def final_lunch(state: workflowState):
    return state

def final_dinner(state: workflowState):
    return state

def final_review(state: workflowState):
    print("\nFinal meal plan:")
    print("Breakfast:", state["breakfast"])
    print("Lunch:", state["lunch"])
    print("Dinner:", state["dinner"])
    return state

# -------- Build Graph --------
graph = StateGraph(workflowState)
memory = MemorySaver()

# Nodes
graph.add_node("get_preferences", get_preferences)
graph.add_node("gen_breakfast", gen_breakfast)
graph.add_node("review_bf", review_bf)
graph.add_node("final_bf", final_bf)

graph.add_node("gen_lunch", gen_lunch)
graph.add_node("review_lunch", review_lunch)
graph.add_node("final_lunch", final_lunch)

graph.add_node("gen_dinner", gen_dinner)
graph.add_node("review_dinner", review_dinner)
graph.add_node("final_dinner", final_dinner)

# Join/final node
graph.add_node("final_review", final_review)

# Edges
graph.add_edge(START, "get_preferences")
graph.add_edge("get_preferences", "gen_breakfast")
graph.add_edge("get_preferences", "gen_lunch")
graph.add_edge("get_preferences", "gen_dinner")

# Review + Finalize per meal
graph.add_edge("gen_breakfast", "review_bf")
graph.add_edge("review_bf", "final_bf")
graph.add_edge("final_bf", "final_review")

graph.add_edge("gen_lunch", "review_lunch")
graph.add_edge("review_lunch", "final_lunch")
graph.add_edge("final_lunch", "final_review")

graph.add_edge("gen_dinner", "review_dinner")
graph.add_edge("review_dinner", "final_dinner")
graph.add_edge("final_dinner", "final_review")

# End
graph.add_edge("final_review", END)

# -------- Compile & Run --------
workflow = graph.compile(checkpointer=memory)

from IPython.display import Image, display

try:
  display(Image(workflow.get_graph().draw_mermaid_png()))
except Exception:
    print(Exception)
    #This requires some extra dependencies and is optional
    pass

config = {"configurable": {"thread_id": "1"}}
result = workflow.invoke({}, config=config, stream_mode="updates")
result

for chunk in workflow.stream({}, config=config, stream_mode="updates"):
    for node_id, value in chunk.items():
        if node_id == "__interrupt__":
            decision = input("Approve or Regenerate? (yes/no): ")
            if decision == "yes":
                workflow.invoke(Command(resume=decision), config=config, stream_mode="updates")
            # else:
            #     goto_node = input("Enter node to go to (e.g., gen_breakfast): ")
            #     workflow.invoke(Command(goto=goto_node), config=config, stream_mode="updates")


ModuleNotFoundError: No module named 'langgraph'